In [9]:
INPUT_VERSION = 18
OUTPUT_VERSION = 3

S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/test01/output/"

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/test01/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017test01/output"

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V%0.2d.pkl" %(INPUT_VERSION)
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"

OUTPUT_FILENAME = "Y2017M09D15_RH_Add_Basin_Data_V%0.2d" %(OUTPUT_VERSION)

In [10]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [11]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [12]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V18.pkl to ../../../../data/test01/input/mergedZonalStatsEE_V18.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V18.csv to ../../../../data/test01/input/mergedZonalStatsEE_V18.csv


In [13]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/test01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [14]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [15]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)

In [16]:
df_ee = pd.read_pickle(inputLocationEE)

In [17]:
df_ee.index.names = ['PFAF_ID']

In [18]:
df_ee.head()

,count_Hybas06,mean_Hybas06,count_area_30s_m2,mean_area_30s_m2,total_area_30s_m2,count_runoff_monthY2014M09,mean_runoff_monthY2014M09,total_volume_runoff_monthY2014M09,count_PLivWN_monthY2014M08,mean_PLivWN_monthY2014M08,...,total_volume_PIrrWW_monthY2014M09,count_runoff_monthY2014M10,mean_runoff_monthY2014M10,total_volume_runoff_monthY2014M10,count_PIndWW_monthY2014M01,mean_PIndWW_monthY2014M01,total_volume_PIndWW_monthY2014M01,count_PLivWW_monthY2014M09,mean_PLivWW_monthY2014M09,total_volume_PLivWW_monthY2014M09
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,2536,111011.0,2536,743658.186761,1.885917e+09,2479,-1.791564e-07,-337.874070,2536,1.392067e-07,...,347598.275922,2479,1.989458e-05,37519.523523,2536,0.002146,4.048026e+06,2536,1.366897e-07,257.785492
111012,3921,111012.0,3921,746186.462653,2.925797e+09,3914,-7.736314e-08,-226.348862,3921,6.749631e-07,...,0.000000,3914,5.720039e-06,16735.674093,3921,0.000229,6.688613e+05,3921,6.624970e-07,1938.331951
111013,1194,111013.0,1194,747422.836265,8.924229e+08,1141,-4.750628e-08,-42.395691,1194,5.224477e-08,...,0.000000,1141,4.924584e-06,4394.811805,1194,0.000694,6.191623e+05,1194,5.123168e-08,45.720323
111014,5605,111014.0,5605,750449.270450,4.206268e+09,5605,0.000000e+00,0.000000,5605,2.158286e-07,...,0.000000,5605,0.000000e+00,0.000000,5605,0.000018,7.558842e+04,5605,2.111498e-07,888.152682
111015,21873,111015.0,21873,758792.279231,1.659706e+10,21086,0.000000e+00,0.000000,21769,2.750456e-08,...,0.000000,21086,2.064860e-08,342.706117,21769,0.000541,8.983459e+06,21769,2.690531e-08,446.549220


In [23]:
sumSeries = df_ee.sum(0)

In [24]:
sumSeries

count_Hybas06                                         2.220802e+08
mean_Hybas06                                          7.239134e+09
count_area_30s_m2                                     2.220802e+08
mean_area_30s_m2                                      1.099506e+10
total_area_30s_m2                                     1.348187e+14
count_runoff_monthY2014M09                            2.111675e+08
mean_runoff_monthY2014M09                             4.722635e+02
total_volume_runoff_monthY2014M09                     4.063999e+12
count_PLivWN_monthY2014M08                            2.214955e+08
mean_PLivWN_monthY2014M08                             1.808036e-01
total_volume_PLivWN_monthY2014M08                     1.459008e+09
count_PLivWW_monthY2014M10                            2.214955e+08
mean_PLivWW_monthY2014M10                             1.791513e-01
total_volume_PLivWW_monthY2014M10                     1.444638e+09
count_PIndWN_monthY2014M09                            2.212373

In [37]:
sumSeries["count_Hybas06"] = -1

In [38]:
test = sumSeries.filter(regex=("count_Hybas06*|count_area_30s_m2")).clip(lower=0)

In [39]:
test

count_Hybas06                0.0
count_area_30s_m2    222080176.0
dtype: float64

In [ ]:
dfTest = df_ee[df_ee["total_volume_reducedmeanrunoff_month_Y1960Y2014M01"] < 0 ]

In [ ]:
mask = df_ee["total_volume_reducedmeanrunoff_month_Y1960Y2014M01"] < 0

df_ee.loc[mask, "total_volume_reducedmeanrunoff_month_Y1960Y2014M01"] = 0

In [ ]:
df_ee.total_volume_reducedmeanrunoff_month_Y1960Y2014M01.min()